# Setup / Package imports
Lets get the necessary packages first :) We will need
* requests to scrape the packages
* pandas for the data handling
* folium for map visualization

In [1]:
import requests
import pandas as pd
import numpy as np
import folium

# Getting the data ready
## Read the webpage

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data = requests.get(url).text

##  Extract and prepare the dataframe
This will be done by
*  selecting only the relevant table
* renaming the columns
* dropping "not assigned" boroughs
* resetting the index to have consistent access
* checking that we do not have "not assigned" neighborhoods (which should be the case after dropping unassigned boroughs)

In [3]:
webpage_data = pd.read_html(html_data)
# we are only interested in the first table
df = webpage_data[0]
# change to american english
df.rename(columns={'Postal Code': 'PostalCode', 'Neighbourhood': 'Neighborhood'}, inplace=True)
# drop unassigned boroughs
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)
# reset the index
df.reset_index(drop=True, inplace= True)
# peek into the data
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
#check if we have unassigned neighborhoods
unassigned_neighborhoods = df.loc[df['Neighborhood'] == "Not assigned"]
if(unassigned_neighborhoods.empty):
    print('No unassigned neighborhoods. OK')
else:
    raise ValueError("There are {} unassigned neighborhood entries".format(unassigned_neighborhoods.shape[0]))

No unassigned neighborhoods. OK


To finish the webscraping and data preparation, we will print out the total number of remaining rows of our dataframe

In [5]:
row_total = df.shape[0]
print('There are {} rows in the prepared dataframe.'.format(row_total))

There are 103 rows in the prepared dataframe.


## Enrich data with latitude and longitude
I didnt manage to get the proxies working for geocoder (sitting behind corporate firewalls), so I chose the easy way and downloaded the .csv file.

Next up, we are going to merge the two tables.

In [6]:
latlong_table = pd.read_csv('Geospatial_Coordinates.csv')
latlong_table.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
toronto_data = pd.merge(df, latlong_table, on='PostalCode')
toronto_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## Sanity check
Looks great so far. I don't  know how they ordered the table on the assignment website. However, lets create the same table using the postal code for our search

In [7]:
# cross check with data from assignment website
codes = ['M5G', 'M2H', 'M4B', 'M1J', 'M4G', 'M4M', 'M1R', 'M9V', 'M9L', 'M5V', 'M1B', 'M5A']
# create an empty dataframe
test_df = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'])
# copy only those entries with specified postal codes
for code in codes:
    test_df = test_df.append(toronto_data.loc[toronto_data['PostalCode'] == code])
# just to make it look the same, lets also reset the index
test_df.reset_index(drop=True, inplace=True)
test_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420


Data seems to be ok. So lets continue :)

# Clustering and visualization
I am going to focus on the boroughs that contain the word Toronto. Therefore, lets first reduce the dataframe.

In [8]:
toronto_boroughs = toronto_data[toronto_data['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_boroughs.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


Since I cannot use geolocator, I looked up the coordinatesand hardcoded them below to invoke a folium map of Toronto.

In [9]:
toronto_lat= 43.70011
toronto_lon= -79.4163
map_toronto = folium.Map(location=[toronto_lat, toronto_lon], zoom_start=10)

In [10]:
# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_boroughs['Latitude'], toronto_boroughs['Longitude'], toronto_boroughs['Borough'], toronto_boroughs['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [11]:
# @hidden_cell
CLIENT_ID = 'test' # your Foursquare ID
CLIENT_SECRET = 'test' # your Foursquare Secret